Laboration för kursen DD1334
Använder SQL-lite för att kommunicera med en databas som sparar data från vårt projekt i kartgenerering

Först måste vi hämta hem map klasser från github och installera dependencies

In [2]:
! pip install depend #fixa sen
%pip install ipywidgets
! git clone git@github.com:poskusen/dd1334.git

Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#fixa': Expected package name at the start of dependency specifier
    #fixa
    ^


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Cloning into 'dd1334'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


Vi börjar med att generera en karta som ska sparas.

In [3]:
from Classes.map import Map
import ipywidgets as widgets
from IPython.display import display, clear_output
from draw_PNG import draw_map
ny_karta = Map(5, 50, 0, 0 ,100, realm_name= 'test_karta') # Skapa en karta med bara byar

draw_map(ny_karta)
output = widgets.Output()
with output: # Visa upp kartan
    image = open("test_image.png", "rb").read() 
    
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=1000,            
        height=1000
    )
    display(image_widget)
display(output)


Output()

Sen connectar vi till databasen och skapar ett nytt table för kartor, continenter och allt content på kontinenterna.

Här uppstår ett stort problem som vi undviker genom att använda pickle.
För att spara vår representation av kontinenten så måste vi spara en lista med punkter. Detta blir väldigt tricky då SQL databaser är baserade på att man sparar 1 värde och inte en lista med värden. Något vi undviker när vi bara serialiserar hela objektet. 
En lösning hade varit skapa ett table per lista och därmed en per kontinent, men det är väldigt bökigt och förstör hela poängen med databasen. Och eftersom vi aldrig vill ha enbart en eller flera punkter från kontinent-listan finns det ingen poäng med att skapa ett helt nytt table, då vi aldrig hämtar enskilda värden från listan. Därför kan man se listan snarare som ett objekt, än en lista. Därför serialiserar vi listan och sparar den sen, allt annat hade varit galenskap.

För att ändå visa att det går gör vi det för städerna.
Vi sparar även bara städerna för floder och berg hade inneburit dubbelarbete, och visar egentligen ingenting.

Den enda fördelen med att köra sql är att det går att hämta kontinenter med rätt 'size' från databasen, dvs att jämföra storleken på serilizationen av points_list.

In [28]:
import sqlite3, csv

conn = sqlite3.connect('database.db')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = OFF;')

tables = ['maps', 'continents', 'villages']
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")

cursor.execute('PRAGMA foreign_keys = ON;')

cursor.execute('''CREATE TABLE maps(
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               map_size_x INT DEFAULT 1000,
               map_size_y INT DEFAULT 1000,
               name VARCHAR(255) NOT NULL,
               amount_continents INTEGER NOT NULL
               );''') # Skapa ett table för kartor

cursor.execute('''CREATE TABLE continents(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                points_serialised VARCHAR(65535),
                id_map INTEGER NOT NULL,
                FOREIGN KEY (id_map) REFERENCES maps(id)
               );''') # Skapa ett table för kontintenter


cursor.execute('''CREATE TABLE villages(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                x INT NOT NULL,
                y INT NOT NULL,
                id_cont INTEGER NOT NULL,
                FOREIGN KEY (id_cont) REFERENCES continents(id)
               );''') # Skapa ett table för byar



Stäng databas om det behövs

In [11]:
conn.close()

Skapa en enkel serialiserings algoritm som kan spara listor och en som kan återskapa innehållet och göra det till en lista

In [33]:
def stringify(continent_list):
    ser = ''
    for point in continent_list:
        str_point = str(point)
        str_point = str_point.strip('()')
        str_point = str_point.replace(' ','')
        ser +=  str_point + ','
    return ser

    

def unstringify(string):
    # Assume `string` is a serialized representation of coordinates, e.g., '522.92,304.12'
    lista_nummer = string.split(',')  # or however your data is structured
    continent_list = []
    for num in lista_nummer:
        # Convert to float, then to int if needed
        continent_list.append((float(num), float(num)))  # Adjust based on actual data structure
    return continent_list




Skapa funktion som extraherar kontinenter, karta och byar och serialisera kontinent_listorna samt lägger till dem och kartan i databasen

In [8]:
def save(map, cursor):
    amount_continents = map.get_continent_amount()
    name_map = map.get_realm_name()
    (x_size, y_size) = map.get_mapsize()
    cursor.execute('''INSERT INTO maps (map_size_x, map_size_y, name, amount_continents)
                    VALUES(?,?,?,?);
                   ''', (x_size, y_size, name_map, amount_continents))
    continents = map.get_continents()
    map_id = cursor.lastrowid
    for cont in continents:
        points_list = cont.get_point_list()
        ser_points = stringify(points_list)
        cursor.execute('''INSERT INTO continents (points_serialised, id_map)
                    VALUES(?,?);
                    ''', (ser_points, map_id))
        villages = cont.get_villages().get_cities_list()
        cont_id = cursor.lastrowid
        for village in villages:
            cursor.execute('''INSERT INTO villages (x, y, id_cont)
                        VALUES(?,?,?);
                           ''', (village[0], village[1], cont_id))

Testa att spara ner en karta

In [29]:

save(ny_karta, cursor)
conn.commit()
cursor.execute("SELECT * FROM continents")
continents = cursor.fetchall()
print("Continents in the database:")
for row in continents:
    print(row)

Continents in the database:
(1, '522.9213017883802,533.7540862496758,567.6765164764042,508.52186074800625,612.1238923223633,520.590110568096,612.9646525512414,521.6002786181066,622.4609846212504,570.4214785482291,655.9702446348869,606.083069538771,669.7130571734431,611.4197335478192,672.9933695775584,614.1178513212004,674.3234634737015,621.2461862607593,661.4397004915628,639.5883282020462,627.2362342854866,658.7355529481905,576.2977992613008,675.5996038093878,553.6305622775233,688.1414192132128,547.5219734087093,689.9684856842662,495.7514429802953,697.5909198453178,485.10118655072847,691.8828096098579,475.6723762550537,683.3588232907082,451.7843090599535,651.0273505512021,434.1952404583692,646.0471947084444,427.3397077198898,641.4413340123821,415.91990935279676,616.6341988216142,445.12017701913663,580.3609311241789,474.1236201426405,549.6582480063789,522.9213017883802,533.7540862496758,', 1)
(2, '647.191885848544,380.76706512651197,669.4288905542091,431.4025578047939,650.2338770101844,

Skapa en funktion som hämtar upp en karta och återskapar kartan

In [10]:
def load(karta_id, cursor): 
    cursor.execute('''SELECT * FROM maps
                JOIN continents ON maps.id = continents.id_map
                JOIN villages ON continents.id = villages.id_cont
                WHERE maps.id = ?
                ;
                ''', (karta_id, )) #Hämta alla kartan med karta_id
    
    data = cursor.fetchall()
    amount_continents = data[4]
    list_continents = []
    for i in range(0, amount_continents): #Hämta kontinenterna
        list_continents.append(unstringify(data[6 + i])) #Avserilisera kontinenterna
    for i in range(6 + amount_continents, len(data)): #Hämta kontinenter

    
    return Map(amount_continents, 100, 0, 0, 50, (data[1], data[2]), data[3], create_new=False, ) #Återskapa kartan
print(load(3, cursor))




IndentationError: expected an indented block after 'for' statement on line 14 (1842703738.py, line 17)

Testa att spara och hämta en karta

In [34]:
from Classes.continent import Continent

#Olle load
def load(karta_id, cursor): 
    continent_cords_list = [] #Every position is a list with all the continents positions
    villages_lists = []

    cursor.execute('''SELECT COUNT(*) 
                      FROM continents 
                      WHERE id_map = ?;''', (karta_id,))
    number_of_continents = cursor.fetchone()[0]  # Count of continents
    
    cursor.execute('''SELECT * 
                  FROM continents 
                  WHERE id_map = ?;''', (karta_id,))#Get all continents
    
    continents = cursor.fetchall()

    for row in continents:
        continent_cords_list.append(unstringify(row[1]))  #Now we have all continent cordinates 

    for continent_id in range (1, number_of_continents+1):
        temp_list = []

        cursor.execute('''SELECT * 
                      FROM villages 
                      WHERE id_cont = ?;''', (continent_id,))
        villages = cursor.fetchall()
        for rows in villages:
            x = rows[1]
            y = rows[2]
            xy = (x,y)
            temp_list.append(xy)
        villages_lists.append(temp_list)

    #Now make continent list containing actual continent instances
    continent_list = []
    for i in range (number_of_continents):

        temp_cont = Continent((1000,1000), 0, 0, 0 , 0, point_list = continent_cords_list[i], content = [None, villages_lists[i]])
        print("HELLO")
        print(villages_lists[i])
        #continent_list.append(temp_cont)

        print(temp_cont.villages)

load(1, cursor)

HELLO
[(522.3364368413961, 550.8612128152554), (524.490405794306, 553.1072061196818), (475.4171974518249, 611.2837087944), (506.7932467803135, 670.5656621798549), (632.1310794877373, 619.7281482742626), (608.5977387390459, 523.3828084526099), (451.42042861747024, 650.2303707407643), (661.4397004915628, 639.5883282020462), (485.10118655072847, 691.8828096098579), (522.9213017883802, 533.7540862496758), (475.6723762550537, 683.3588232907082), (655.9702446348869, 606.083069538771), (576.2977992613008, 675.5996038093878), (474.1236201426405, 549.6582480063789)]


AttributeError: 'Continent' object has no attribute 'villages'

Testa att spara och hämta flera kartor